In [0]:

from pyspark.sql.functions import date_format

df =spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where("(process_date between '2020-02-24' and '2020-02-26') and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").cache()

df = df.withColumn("time",df["time"].cast('date'))
new_df = df.withColumn("event_month",date_format('time', 'yyyy-MM'))
result =  new_df.groupBy("event_month").agg({"*":"count"}).cache().collect()


print result


In [0]:

print result[0]
spark.createDataFrame(result).withColumnRenamed("count(1)","agg_count").coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/event_time_repartition_streaming_20200220_20200310_count/")

# result.coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/streaming_20200220_20200310_count/")




In [0]:

from aadatapipelinecore.core.log import logger
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import date_format


# df = spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/').parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={hourly,daily,weekly}/process_date={2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24}").where("market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").cache()
df =spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where("(process_date between '2020-02-20' and '2020-03-09') and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").cache()
df = df.withColumn("time",df["time"].cast('date'))
new_df = df.withColumn("event_month",date_format('time', 'yyyy-MM'))
result =  new_df.groupBy("event_month").agg({"*":"count"}).cache().collect()



In [0]:

review_v4 = df.withColumn("event_month",date_format('time', 'yyyy-MM')).filter("event_month='2013-05'").select("review_id","_identifier").cache()
# df.withColumn("event_month",date_format('time', 'yyyy-MM')).filter("event_month='2013-05'").groupBy("_identifier").agg({"*":"count"}).orderBy(df._identifier.desc()).show()

In [0]:



load_temp= spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_temp/review_load_temp_streaming_20200220_20200310/event_month=2013-05").filter("process_date != '2020-03-10'").select("review_id","_identifier").cache()

In [0]:

print load_temp.select("review_id").distinct().count()
print review_v4.select("review_id").distinct().count()


In [0]:

import pyspark.sql.functions as F

review_v4.createOrReplaceTempView("raw")
load_temp.createOrReplaceTempView("unified")
result_test = spark.sql("select count(_identifier), _identifier from (select review_id,_identifier from unified EXCEPT select review_id,_identifier from raw) as prod group by _identifier order by _identifier desc ")
# df.name.rlike
# print result_test.filter(result_test._identifier.rlike("220200310*") ).show(30,False)
result_test.show()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.review/



In [0]:

from pyspark.sql.utils import AnalysisException

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get


s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_agg_v3/", depth_is_1=True)
print len(path_list)

def get_month_count(path):
    df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).select("review_id")
    month=path.split("=")[1].replace("/","")
    try:
        df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date={}*/".format(month)).filter("market_code in ('apple-store','google-play')").select("review_id")
        return month, df1.union(df2).select("review_id").distinct().count()
    except AnalysisException:
        print 'no monthly data'.format(month)
        return month, df1.select("review_id").distinct().count()
        

# df.coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/historical_data")


def es_doc(month):
    month = month.replace("-","")
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp", "C38vEJEuraCw")
    ios_url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-*-{}/_count".format(
        month)
    ios_resp = plain_get(ios_url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    ios_doc = json.loads(ios_resp.text)
    return ios_doc["count"]


for path in path_list[::-1]:
    month, unified_count = get_month_count(path)
    spark.createDataFrame(result).withColumnRenamed("count(1)","agg_count").coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/streaming_20200220_20200310_count/")

    
    # db_count = es_doc(month)
    # if unified_count!= db_count:
    #     print 'not equal!!!! month {}, unified count is {}, db count is {}, db-unified diff is {} '.format(month, unified_count, db_count, db_count-unified_count)
    # else:
    #     print 'month {}, unified count is {}, db count is {}'.format(month, unified_count, db_count)

In [0]:

import pyspark.sql.functions as F

df = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date.v3/date=2019-01-14").withColumn("id_1", F.monotonically_increasing_id()).cache().filter("_identifier = '220200218115410210'")
print df.take(20)

In [0]:

import pyspark.sql.functions as F

# new_df = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date.v3/date=2020-02*").withColumn("id_1", F.monotonically_increasing_id()).cache().filter("id_1 = '3796751090178'")
new_df.filter("id_1='3796751090127'")
print new_df.take(20)

In [0]:

import pyspark.sql.functions as F

df = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date.v3/date=2020-02-*").withColumn("id_1", F.monotonically_increasing_id()).cache().filter("id_1 = '3994319585282'").show(20,False)

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_month.v3/
 


In [0]:


df = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date=2019-12-01/0-11").filter("index is null")
print df.count()


In [0]:


from aadatapipelinecore.core.log import logger
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date.v3/date={}/'
s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = ['process_granularity={hourly, daily, weekly}/process_date=%s/' % item for item in ["2020-02-11"]]
print path_list[0]
for path in path_list:
    print path.split("/")[1].split("=")[1]
    count_1 = spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/').parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/{}".format(path)).where("market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").select("review_id").distinct().count()
    df = spark.read.json(dest_path.format(path.split("/")[1].split("=")[1])).cache()
    count_2 = df.filter("index is not null").count()
    count_3 = df.filter("_identifier is not null").count()
    print count_1, count_2, count_3
    if count_1 == count_2 == count_3 :
        print 'pass', path
    else:
        print "not equal!!!!!!", count_1 , count_2, count_3,  path


In [0]:

#int-ss-review_v1-google-play-android-all-201901


def es_doc():
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-*-200808*/_count"
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    print doc
    
es_doc()


In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

# from aadatapipelinecore.core.fs.device import meta_bucket


s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_d = s3_bucket_list.all(prefix="review-uniform/DAILY/", depth_is_1=True)
date_list_raw = [x.split("/")[2] for x in path_d]

s3_bucket_list_unified = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_d_unified = s3_bucket_list_unified.all(prefix="unified/review.v1/fact/process_granularity=daily/", depth_is_1=True)
date_list_unified = [x.split("/")[4].split("=")[1] for x in path_d_unified]

not_exist_data_set = sorted(set(date_list_raw) - set(date_list_unified))
logger.info('Need to check !!! date is not existed!!!!! {}'.format(not_exist_data_set))

for x in not_exist_data_set:
    el = "review-uniform/DAILY/{}/".format(x)
    path_d.remove(el)
print path_d[899]
# logger.info('test date list {}'.format(path_d))

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

# compared the data from uniform(raw data) to unified data
raw_path_d = "s3://prod_appannie_uniform_public_app_store_data/"

unified_path_d = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date={unified_process_date}/process_hour={unified_process_hour}/device_code={unified_device_code}/market_code={unified_market_code}"


def count_raw_unified_data_daily():
    for p in path_d[899:900]:
        process_hour_raw_path = s3_bucket_list.all(p, depth_is_1=True)
        for x in process_hour_raw_path:
            # logger.info('raw path with process hour {}'.format(x))
            platforms_process_hour_raw_path_list = s3_bucket_list.all(x, depth_is_1=True)
            for raw_path in platforms_process_hour_raw_path_list:
                raw_count_sum, root_raw_data_path = calculate_sum_data(raw_path)
                # logger.info('raw_count_sum : {}'.format(raw_count_sum))
                global unified_count

                unified_count = 0
                if raw_count_sum == 0:
                    continue
                upath = unified_path_d.format(unified_process_date=root_raw_data_path.split("/")[2],
                                              unified_device_code=platforms[root_raw_data_path.split("/")[4]][
                                                  "device_code"],
                                              unified_market_code=platforms[root_raw_data_path.split("/")[4]][
                                                  "market_code"],
                                              unified_process_hour=str(int(root_raw_data_path.split("/")[3])))

                try:
                    unified_count = spark.read.parquet(upath).count()
                    print 'raw_count_sum : {} , unified_count : {} ,unified path is : {} '.format(
                            raw_count_sum,
                            unified_count, root_raw_data_path)

                except AnalysisException as e:
                    print 'the path is not exist for unified level!!! the raw path is {}, unified path is {}'.format(
                        raw_path_d + root_raw_data_path, upath)

            # logger.info( 'unified_count', unified_count

                if raw_count_sum != unified_count:
                    print 'count is not equal!!! the raw path is {}, unified path is {} , raw count is : {}, unified count is : {}'.format(
                            raw_path_d + root_raw_data_path, upath,
                            raw_count_sum, unified_count)


def calculate_sum_data(deep_raw_path):
    logger.info(deep_raw_path)
    raw_count = spark.read.csv(raw_path_d + deep_raw_path, sep="\t").count()
    return raw_count, deep_raw_path


count_raw_unified_data_daily()


In [0]:

from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import types
platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}


# get current path list
# path=[]
# for object_summary in my_bucket.objects.filter(Prefix="review-uniform/MONTHLY/"):
#    path.append(object_summary.key[:object_summary.key.rfind("/")])

# path = list(dict.fromkeys(path))
# print path

# compared the data from uniform(raw data) to unified data
raw_path = "s3://prod_appannie_uniform_public_app_store_data/"
unified_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date={unified_process_date}/process_hour=23/device_code={unified_device_code}/market_code={unified_market_code}"




s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_h = s3_bucket_list.all(prefix="review-uniform/MONTHLY/2016-08-31/", depth_is_1=True)

print path_h

def check_raw_unified_data_equals():
    t = unittest.TestCase('run')

    for p in path_h:
        raw_result = spark.read.csv(raw_path + p, sep="\t").selectExpr("_c0 as id", "_c1 as product_id",
                                                                       "_c2 as product_version", "_c3 as user_id",
                                                                       "_c4 as user_name", "_c5 as user_language",
                                                                       "_c6 as user_device", "_c7 as user_purchased",
                                                                       "_c8 as rating", "_c9 as country",
                                                                       "_c10 as date", "_c11 as language",
                                                                       "_c12 as title", "_c13 as title_t",
                                                                       "_c14 as content", "_c15 as content_t",
                                                                       "_c16 as reply", "_c17 as reply_t",
                                                                       "_c18 as reply_date").cache().take(1)
        print raw_result
        check_id = raw_result[0]["id"]

        upath = unified_path.format(unified_process_date=p.split("/")[2],
                                    unified_device_code=platforms[p.split("/")[3]]["device_code"],
                                    unified_market_code=platforms[p.split("/")[3]]["market_code"])
        unified_result = spark.read.parquet(upath).filter("review_id='{test_id}'".format(test_id=check_id)).take(2)
         
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date"
                      }

        for key, value in column_dic.items():
            check_value(key, raw_result[0][key], unified_result[0][value], p)


def check_value(index, exp, act, p):
    if exp:
        if index == 'rating' or index == "product_id" :
            exp = int(exp)
            act= int(act)
            if exp == act:
                pass
                # print 'equals'
            if exp != act:
                print 'number NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act)
        elif index == 'user_purchased':
            exp = str(exp).lower()
            act= str(act).lower()
            if exp == act:
                pass
                # print 'equals'
            if exp != act:
                print 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act)
        else:
            if exp.lower() == act.lower():
                pass
                # print 'equals'
            if exp != act:
                print 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act)


check_raw_unified_data_equals()


In [0]:

from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import types
platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}


raw_path = "s3://prod_appannie_uniform_public_app_store_data/"
unified_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=weekly/process_date={unified_process_date}/process_hour=23/device_code={unified_device_code}/market_code={unified_market_code}"



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_h = s3_bucket_list.all(prefix="review-uniform/WEEKLY/2018-12-11/23/", depth_is_1=True)

print path_h

def check_raw_unified_data_equals_weekly():
    t = unittest.TestCase('run')
    
    for p in path_h:
        df = spark.read.csv(raw_path + p, sep="\t").take(1)
        print '#################################### lenght of columns', len(df.columns)
        raw_result = df.selectExpr("_c0 as id", "_c1 as product_id",
                                                                       "_c2 as product_version", "_c3 as user_id",
                                                                       "_c4 as user_name", "_c5 as user_language",
                                                                       "_c6 as user_device", "_c7 as user_purchased",
                                                                       "_c8 as rating", "_c9 as country",
                                                                       "_c10 as date", "_c11 as language",
                                                                       "_c12 as title", "_c13 as title_t",
                                                                       "_c14 as content", "_c15 as content_t",
                                                                       "_c16 as reply", "_c17 as reply_t",
                                                                       "_c18 as reply_date").cache().take(1)
        print raw_result
        check_id = raw_result[0]["id"]

        upath = unified_path.format(unified_process_date=p.split("/")[2],
                                    unified_device_code=platforms[p.split("/")[4]]["device_code"],
                                    unified_market_code=platforms[p.split("/")[4]]["market_code"])
        unified_result = spark.read.parquet(upath).filter("review_id='{test_id}'".format(test_id=check_id)).take(2)
         
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date"}#,"userreview_url":"userreview_url","reply_id":"reply_id"
                   #   }

        for key, value in column_dic.items():
            check_value(key, raw_result[0][key], unified_result[0][value], p)


def check_value(index, exp, act, p):
    print index, exp, act, p
    if exp:
        if index == 'rating' or index == "product_id" :
            exp = int(exp)
            act= int(act)
            if exp == act:
                pass
                # print 'equals'
            if exp != act:
                print 'number NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act)
        elif index == 'user_purchased':
            exp = str(exp).lower()
            act= str(act).lower()
            if exp == act:
                pass
                # print 'equals'
            if exp != act:
                print 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act)
        else:
            if exp.lower() == act.lower():
                pass
                # print 'equals'
            if exp != act:
                print 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act)


check_raw_unified_data_equals_weekly()


In [0]:


df1= spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/MONTHLY/2009-08-31/gp",sep="\t").cache()
df2 =spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date=2009-08-31/process_hour=23/device_code=android-all/").cache()

print df2.join(df1, df1._c0==df2.review_id, 'outer' ).filter("_c0 is null").count()


In [0]:

import unittest
import boto3
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3 
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_d= s3_bucket_list.all(prefix="review-uniform/DAILY/", depth_is_1=True )
path_d = path_d[8:]
# print path_d


platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

# s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2017-06-30/00/wp/40700000020383.gz/
# compared the data from uniform(raw data) to unified data
raw_path_d="s3://prod_appannie_uniform_public_app_store_data/"
unified_path_d="s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date={unified_process_date}/process_hour={unified_process_hour}/device_code={unified_device_code}/market_code={unified_market_code}"

def count_raw_unified_data_daily():
    t = unittest.TestCase('run')
    unified_count=0
    for p in path_d:

        # print data_path
        hour_raw_path=s3_bucket_list.all(p, depth_is_1=True )
        print hour_raw_path
        hour_raw_path = ['review-uniform/DAILY/2019-12-02/23/ios/']
        print hour_raw_path
        for x in hour_raw_path:
            print x
            deep_raw_path=s3_bucket_list.all(x, depth_is_1=True )
            # deep_raw_path = "review-uniform/DAILY/2019-12-02/23/ios/"
            raw_count_sum , dp_path = calculate_sum_data(deep_raw_path)

            print  'dp_path', dp_path   
            upath=unified_path_d.format(unified_process_date=dp_path.split("/")[2],unified_device_code=platforms[dp_path.split("/")[4]]["device_code"],unified_market_code=platforms[dp_path.split("/")[4]]["market_code"],unified_process_hour=dp_path.split("/")[3])
            print 'unified path is : ', upath
            try:
                unified_count = spark.read.parquet(upath).count()
            except AnalysisException as e:
                print 'the path is not exist for unified level!!! the path is', raw_path_d+p, upath
        
        # print 'unified_count', unified_count

            if raw_count_sum != unified_count:
                print 'count is not equal!!! the path is : ', raw_path_d+p, upath, "raw is: {} , unified is : {} ".format(raw_count_sum, unified_count)


def calculate_sum_data(deep_raw_path):
    print deep_raw_path
    count_one_day=0
    for dp in deep_raw_path:
        unified_count=0
        print 'raw_path is: ', raw_path_d+dp
        if 'wp' in raw_path_d+dp:
            raw_count = spark.read.csv(raw_path_d+dp,sep="\t").count()
            count_one_day = count_one_day+raw_count
        # print 'raw_count', raw_count
            if raw_count == 0:
            # print 'raw is empty, path is : ' , raw_path+p
                continue
        
    print "calculate sum data", deep_raw_path[0] 
    print deep_raw_path[0].split("/")[3]
    print 'count is : ', count_one_day
    return count_one_day, deep_raw_path[1]

count_raw_unified_data_daily()

# calculate_sum_data(["review-uniform/DAILY/2019-12-02/23/ios/"])


In [0]:

import unittest
import boto3

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('prod_appannie_uniform_public_app_store_data')


# get current path list
#path=[]
#for object_summary in my_bucket.objects.filter(Prefix="review-uniform/MONTHLY/"):
#    path.append(object_summary.key[:object_summary.key.rfind("/")])

#path = list(dict.fromkeys(path))
# print path

# compared the data from uniform(raw data) to unified data
raw_path="s3://prod_appannie_uniform_public_app_store_data/"
unified_path="s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date={unified_process_date}/process_hour=23/device_code={unified_device_code}/market_code={unified_market_code}"

def check_raw_unified_data_equals():
    t = unittest.TestCase('run')
    # get current path list
    path=[]
    for object_summary in my_bucket.objects.filter(Prefix="review-uniform/MONTHLY/"):
        path.append(object_summary.key[:object_summary.key.rfind("/")])

    path = list(dict.fromkeys(path))

    for p in path:
        raw_result = spark.read.csv(raw_path+p,sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as rating", "_c9 as country", "_c10 as date","_c11 as language", "_c12 as title", "_c13 as title_t", "_c14 as content", "_c15 as content_t", "_c16 as reply", "_c17 as reply_t", "_c18 as reply_date").cache().take(1)
    
        check_id = raw_result[0]["id"]
    
    
        upath=unified_path.format(unified_process_date=p.split("/")[2],unified_device_code=platforms[p.split("/")[3]]["device_code"],unified_market_code=platforms[p.split("/")[3]]["market_code"] )
        result= spark.read.parquet(upath).filter("review_id='{test_id}'".format(test_id=check_id)).take(2)
        check_value('1', raw_result[0]["product_id"] , str(result[0]["app_id"]), p)
        check_value('2', raw_result[0]["product_version"], result[0]["product_version"], p )
        check_value('3',raw_result[0]["user_id"], result[0]["user_id"], p)
        check_value( '4',raw_result[0]["user_name"],  result[0]["user_name"], p )
        check_value('5', raw_result[0]["user_language"] , result[0]["user_language"], p)
        check_value('6', raw_result[0]["user_device"],result[0]["user_device"], p)
        check_value('7', raw_result[0]["user_purchased"].lower() ,str(result[0]["user_purchased"]).lower(), p)
        check_value('8', raw_result[0]["rating"] ,str(result[0]["rating"]), p)
        check_value('9', raw_result[0]["country"] , str(result[0]["country_code"]), p)
        check_value('10', raw_result[0]["date"], result[0]["time"], p)
        check_value('11', raw_result[0]["title"] ,str(result[0]["title"]), p)
        check_value('12', raw_result[0]["content"] ,result[0]["content"],p)
        check_value('13', raw_result[0]["reply"] ,result[0]["reply"], p)
        check_value('14', str(raw_result[0]["reply_date"]), str(result[0]["reply_date"]), p)

def check_value(index, exp,act,p):
   if exp != act:
       print 'index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act)


# def check_raw_unified_data_are_equals():
#     t = unittest.TestCase('run')
#     for p in path:
#         raw_count = spark.read.csv(raw_path+p,sep="\t").count()
#         # print raw_path+p
#         # print 'raw_count', raw_count
#         if raw_count == 0:
#             # print 'raw is empty, path is : ' , raw_path+p
#             continue
        
#         upath=unified_path.format(unified_process_date=p.split("/")[2],unified_device_code=platforms[p.split("/")[3]]["device_code"],unified_market_code=platforms[p.split("/")[3]]["market_code"] )
#         try:
#             unified_count = spark.read.parquet(upath).count()
#         except AnalysisException as e:
#             print 'the path is not exist for unified level!!! the path is', raw_path+p, upath
        
#         # print 'unified_count', unified_count

#         if raw_count != unified_count:
#             print 'count is not equal!!! the path is : ', raw_path+p, upath, "raw is: {} , unified is : {} ".format(raw_count, unified_count)


#     print raw_result[0]
#     check_id = raw_result[0]["id"]

#     result= spark.read.parquet(upath).filter("review_id='{test_id}'".format(test_id=check_id)).take(2)
#     print result
    
#     print raw_result[0]["product_id"] == str(result[0]["app_id"])
#     print raw_result[0]["product_version"] == result[0]["product_version"]
#     print raw_result[0]["user_id"] == result[0]["user_id"]
#     print raw_result[0]["user_name"] == result[0]["user_name"]
#     print raw_result[0]["user_language"] == result[0]["user_language"]
#     print raw_result[0]["user_device"] == result[0]["user_device"]
#     print raw_result[0]["user_purchased"].lower() == str(result[0]["user_purchased"]).lower()
#     print raw_result[0]["rating"] == str(result[0]["rating"])
#     print raw_result[0]["country"] == str(result[0]["country_code"])
#     print raw_result[0]["date"] == result[0]["time"]
#     print raw_result[0]["title"] == str(result[0]["title"])
#     print raw_result[0]["content"] == result[0]["content"]
#     print raw_result[0]["reply"] == result[0]["reply"]
#     print str(raw_result[0]["reply_date"]) == str(result[0]["reply_date"])

#     # print raw_count[0]["product_id"] = result
    
    # print raw_path+p
    # print 'raw_count', raw_count
    #if raw_count == 0:
    #    print 'raw is empty, path is : ' , raw_path+p
        
    #upath=unified_path.format(unified_process_date=p.split("/")[2],unified_device_code=platforms[p.split("/")[3]]["device_code"],unified_market_code=platforms[p.split("/")[3]]["market_code"] )
    #unified_count = spark.read.parquet(upath).count()
    # print 'unified_count', unified_count

    #if raw_count != unified_count:
    #    print 'path is : ', raw_path+p, upath, "raw is: {} , unified is : {} ".format(raw_count, unified_count)



check_raw_unified_data_equals()


In [0]:
%python
import boto3
from aadatapipelinecore.core.fs.device import s3 
from aadatapipelinecore.core.fs import Conf


s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
l =  s3_bucket_list.all(prefix="review-uniform/DAILY/", depth_is_1=True )
print l[12:]



In [0]:


print spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/MONTHLY/2014-03-31/amazon",sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as rating", "_c9 as country", "_c10 as date","_c11 as language", "_c12 as title", "_c13 as title_t", "_c14 as content", "_c15 as content_t", "_c16 as reply", "_c17 as reply_t", "_c18 as reply_date").filter("id='067dc71871af99bcc6050253761eab0e'").show()
print '####################'
print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date=2014-03-31/process_hour=23/").filter("review_id='067dc71871af99bcc6050253761eab0e'").show(20,False,vertical=True)


# raw_path is:  s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2017-06-08/00/wp/
# unified path is :  s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-06-08/process_hour=00/device_code=windows-phone/market_code=windows-store
# the path is not exist for unified level!!! the path is s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2017-06-08/ s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-06-08/process_hour=00/device_code=windows-phone/market_code=windows-store


In [0]:

# print spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/MONTHLY/2017-06-30/amazon/part-00000",sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as userreview_url", "_c9 as rating", "_c10 as country","_c11 as date", "_c12 as language", "_c13 as title", "_c14 as title_t", "_c15 as content", "_c16 as content_t", "_c17 as reply_id", "_c18 as reply", "_c19 as reply_t", "_c20 as reply_date").filter("id='gp:AOqpTOFEMK10tMaetYLgLXvtiQaWlmepbOWKdNtQ9QRZz5acpAeEcYl9mVnxuN9rdJA0wxLUOJa5NviecYCPmg'").show(10,False, vertical=False)
print spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/MONTHLY/2017-06-30/amazon/part-00000",sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as rating", "_c9 as country", "_c10 as date","_c11 as language", "_c12 as title", "_c13 as title_t", "_c14 as content", "_c15 as content_t", "_c16 as reply", "_c17 as reply_t", "_c18 as reply_date").cache().filter("id='R17N52YEZW6F65'").show(20,False, vertical=True)
print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date=2017-06-30").cache().filter("review_id='R17N52YEZW6F65'").show(20,False,vertical=True)



In [0]:
%%sh
aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/WEEKLY/2018-12-10/23/ios/ |head -3
# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-06-14/process_hour=14/device_code=windows-phone/market_code=windows-store/



# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/ # --recursive --human --summarize | tail -30
# aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/HOURLY/2017-07-01/00/gp/20600007065190.gz  --recursive --human --summarize | tail -30
# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=hourly/process_date=2017-07-01/process_hour=23/device_code=android-all/market_code=google-play/ --recursive --human --summarize | tail -30

# 3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=hourly/process_date=2017-07-01/process_hour=23/device_code=android-all/market_code=google-play/
# #aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-06-10/process_hour=23/device_code=android-all/market_code=google-play/
# review-uniform/DAILY/2017-05-30/


In [0]:


df1= spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2018-09-09/00/amazon/",sep="\t").cache()
print df1.count()
df2 =spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2018-09-09/process_hour=0/device_code=android-all/market_code=amazon-store/").cache()
print df2.count()
print df2.join(df1, df1._c0==df2.review_id, 'outer' ).filter("review_id is null").count()
# 2020-02-01 02:03:23,278 - INFO - MainThread - aadatapipelinecore - count_raw_unified_data_daily - 79 - count is not equal!!! the raw path is s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2018-09-09/00/amazon/, unified path is s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2018-09-09/process_hour=0/device_code=android-all/market_code=amazon-store , raw count is : 5540362, unified count is : 5538749


In [0]:

print spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2017-07-30/14/win/40600000049538.gz/",sep="\t").show()

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-07-30/process_hour=0/device_code=android-all/market_code=amazon-store").count()

In [0]:
%%sh
echo 'raw:'
aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2017-06-22/14/win/40600000049538.gz
 
echo 'unified:'
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-06-22/process_hour=1/


In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_d = s3_bucket_list.all(prefix="review-uniform/DAILY/", depth_is_1=True)
date_list_raw = [x.split("/")[2] for x in path_d]

s3_bucket_list_unified = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_d_unified = s3_bucket_list_unified.all(prefix="unified/review.v1/fact/process_granularity=daily/", depth_is_1=True)
date_list_unified = [x.split("/")[4].split("=")[1] for x in path_d_unified]

not_exist_data_set = sorted(set(date_list_raw) - set(date_list_unified))
# logger.info('Need to check !!! date is not existed!!!!! {}'.format(not_exist_data_set))

for x in not_exist_data_set:
    el = "review-uniform/DAILY/{}/".format(x)
    path_d.remove(el)
# logger.info('test date list {}'.format(path_d))

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

# compared the data from uniform(raw data) to unified data
raw_path_d = "s3://prod_appannie_uniform_public_app_store_data/"

unified_path_d = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date={unified_process_date}/process_hour={unified_process_hour}/device_code={unified_device_code}/market_code={unified_market_code}"


def calculate_sum_data(deep_raw_path):
    logger.info("deep_raw_path {}".format(deep_raw_path))
    count_one_day = 0
    # for dp in deep_raw_path:
        # raw_count = spark.read.csv(raw_path_d + dp, sep="\t").count()
    count_one_day = count_one_day + 1
        # logger.info('count for one day {}'.format(count_one_day))
        # logger.info('deep_raw_path[0] is {}'.format(deep_raw_path[0]))
        # logger.info('dp is : {} '.format(dp))

    return count_one_day, deep_raw_path


def count_raw_unified_data_daily():
    for p in path_d:
        process_hour_raw_path = s3_bucket_list.all(p, depth_is_1=True)
        for x in process_hour_raw_path:
            logger.info('raw path with process hour {}'.format(x))
            platforms_process_hour_raw_path_list = s3_bucket_list.all(x, depth_is_1=True)
            for raw_path in platforms_process_hour_raw_path_list: 
                print "items in platforms_process_hour_raw_path_list" , raw_path
                # raw_count_sum, root_raw_data_path = calculate_sum_data(raw_path)
                # logger.info('root_raw_data_path : {}'.format(raw_path))
                # logger.info('raw_count_sum : {}'.format(raw_count_sum))
            
count_raw_unified_data_daily()


In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_d = s3_bucket_list.all(prefix="review-uniform/DAILY/", depth_is_1=True)
date_list_raw = [x.split("/")[2] for x in path_d]

s3_bucket_list_unified = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_d_unified = s3_bucket_list_unified.all(prefix="unified/review.v1/fact/process_granularity=daily/", depth_is_1=True)
date_list_unified = [x.split("/")[4].split("=")[1] for x in path_d_unified]

not_exist_data_set = sorted(set(date_list_raw) - set(date_list_unified))
# logger.info('Need to check !!! date is not existed!!!!! {}'.format(not_exist_data_set))

for x in not_exist_data_set:
    el = "review-uniform/DAILY/{}/".format(x)
    path_d.remove(el)
print path_d[699]

In [0]:

from elasticsearch import Elasticsearch
import time
import datetime
from pyspark.sql import Row


def get_connection():
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/"],http_auth=("bdp_qa","6trmJcZBJrKWdtN2"),port=19200)
    return es_connection
con_es=get_connection()
query='''{"query":{"bool":{"must":[{"match":{"market_code":"apple-store"}}]}},"aggs":{"review_id":{"cardinality":{"field":"_id"}}},"size":1,"sort":[{"date":{"order":"desc"}}]}'''
def get_result(query):
    result = con_es.search(index="int-ss-review_v2-apple-store-ios-all-201703", body=query)
    print result["hits"]["total"]["value"]

get_result(query)